In [ ]:
! pip install folium
! pip install pycountry
! curl https://hitchmap.com/dump.sqlite > points.sqlite

In [ ]:
import folium
import sqlite3
import pandas as pd
from tqdm import tqdm
import numpy as np
import pycountry
import requests

In [ ]:
fn = 'points.sqlite'
points = pd.read_sql('select * from points where not banned', sqlite3.connect(fn))

In [ ]:
country_wait = points.groupby('country')['wait'].mean().reset_index().rename(columns={'wait': 'avg_wait'})
len(country_wait)


In [ ]:
country_wait["avg_wait"] = np.clip(country_wait["avg_wait"], 0, 100).tolist()

In [ ]:
def alpha2_to_alpha3(alpha2_code):
    try:
        country = pycountry.countries.get(alpha_2=alpha2_code.upper())
        return country.alpha_3 if country else None
    except:
        return None

In [ ]:
country_wait["code"] = country_wait["country"].apply(alpha2_to_alpha3)

In [ ]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_json_data = requests.get(url).json()

# Create a folium map
m = folium.Map(location=[20, 0], zoom_start=2)

# Add the Choropleth
folium.Choropleth(
    geo_data=geo_json_data,
    name='choropleth',
    data=country_wait,
    columns=['code', 'avg_wait'],
    key_on='feature.id',
    fill_color='RdYlGn_r',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color="white",
    bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    legend_name='Average Waiting Time',
).add_to(m)

# Optional: add layer control
folium.LayerControl().add_to(m)

# Save screenshot
m

In [ ]:
# from https://data.worldbank.org/indicator/EN.POP.DNST?name_desc=false
pop_density = pd.read_csv("API_EN.POP.DNST_DS2_en_csv_v2_85433.csv", header=2)
pop_density.head(1)

In [ ]:
country_samples = pd.DataFrame(points["country"].value_counts())
country_samples["code"] = country_samples.index.map(alpha2_to_alpha3)
country_samples = pd.merge(country_samples, pop_density[["Country Code", "2022"]], left_on="code", right_on="Country Code", how="left")
country_samples["normalized_count"] = country_samples["count"] / country_samples["2022"]
country_samples = country_samples.sort_values(by="normalized_count", ascending=False)
len(country_samples)

In [ ]:
country_samples.head(20)

In [ ]:
# from https://data.worldbank.org/indicator/SP.POP.TOTL
pop_total = pd.read_csv("API_SP.POP.TOTL_DS2_en_csv_v2_131993.csv", header=2)
pop_total.head(1)

In [ ]:
country_samples = pd.DataFrame(points["country"].value_counts())
country_samples["code"] = country_samples.index.map(alpha2_to_alpha3)
country_samples = pd.merge(country_samples, pop_total[["Country Code", "2023"]], left_on="code", right_on="Country Code", how="left")
country_samples["normalized_count"] = country_samples["count"] / country_samples["2023"]
country_samples = country_samples.sort_values(by="normalized_count", ascending=False)
len(country_samples)

In [ ]:
country_samples.head(30)

In [ ]:
# from https://data.worldbank.org/indicator/AG.LND.TOTL.K2
size = pd.read_csv("API_AG.LND.TOTL.K2_DS2_en_csv_v2_85432.csv", header=2)
size.head(1)

In [ ]:
country_samples = pd.DataFrame(points["country"].value_counts())
country_samples["code"] = country_samples.index.map(alpha2_to_alpha3)
country_samples = pd.merge(country_samples, size[["Country Code", "2022"]], left_on="code", right_on="Country Code", how="left")
country_samples["normalized_count"] = country_samples["count"] / country_samples["2022"]
country_samples = country_samples.sort_values(by="normalized_count", ascending=False)
country_samples["normalized_count"] = np.clip(country_samples["normalized_count"], 0, 0.08).tolist()
len(country_samples)

In [ ]:
country_samples.head(30)

In [ ]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_json_data = requests.get(url).json()

# Create a folium map
m = folium.Map(location=[20, 0], zoom_start=2)

# Add the Choropleth
folium.Choropleth(
    geo_data=geo_json_data,
    name='choropleth',
    data=country_samples,
    columns=['code', 'normalized_count'],
    key_on='feature.id',
    fill_color='YlOrBr',
    fill_opacity=1.0,
    line_opacity=0.2,
    nan_fill_color="white",
    bins=20,
    legend_name='Samples / km^2',
).add_to(m)

# Optional: add layer control
folium.LayerControl().add_to(m)

# Save screenshot
m